In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import pandas as pd
from IPython.display import HTML
import sys

project_root = os.path.dirname(os.getcwd())
sys.path.append(project_root)

from src.classifiers import is_uae_real_estate_query, llm_classifier
from src.tools import visualize_tool, safe_dataframe_tool, extract_data_intent
from src.geo_tools import generate_google_maps_html

load_dotenv()

True

In [2]:
result =  safe_dataframe_tool("Retrieve properties located within a 2-mile radius of Dubai Eye.")

/tmp/ipykernel_64253/2055471567.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result =  safe_dataframe_tool("Retrieve properties located within a 2-mile radius of Dubai Eye.")


Dataset loaded successfully.


In [4]:
extract_data_intent("Retrieve properties located within a 2-mile radius of Dubai Eye.")

'Extract data of properties located within a 2-mile radius of Dubai Eye.'

In [5]:
from src.tools import load_pandas_ai_dataframe

hi = load_pandas_ai_dataframe()

Dataset loaded successfully.


In [7]:
result = hi.chat("Extract data of properties located within a 2-mile radius of Dubai Eye.")

In [19]:
import json
from typing import Dict, Any
from langchain.tools import tool

DESCRIPTION_SELECT_CALCULATE = "Executes a query on the UAE real estate dataset using pandas-ai and returns the result."

@tool(description=DESCRIPTION_SELECT_CALCULATE)
def safe_dataframe_tool(query: str) -> str:
    """
    Executes a natural language query on the UAE real estate dataset.
    Returns a JSON-formatted string.
    """
    try:
        data = load_pandas_ai_dataframe()  
        result = data.chat(query).to_dict()

        if result["error"] is None:
            return json.dumps({
                "result": result["value"].to_dict(orient="records"),
                "success": True,
                "error": None
            })
        else:
            return json.dumps({
                "error": result["error"],
                "success": False,
                "solution": "Check your query syntax or the dataset structure."
            })

    except Exception as e:
        return json.dumps({
            "error": str(e),
            "success": False,
            "solution": "Check your query syntax or the dataset structure."
        })


In [20]:
result = safe_dataframe_tool("Extract data of properties located within a 2-mile radius of Dubai Eye.")

Dataset loaded successfully.


In [21]:
result

'{"result": [{"title": "Dorchester Managed 3BR Waterfront Resale", "displayAddress": "Orla by Omniyat, Palm Jumeirah, Dubai", "bathrooms": 4.0, "bedrooms": 3.0, "addedOn": "2024-07-29 08:02:12+00:00", "type": "Residential for Sale", "price": 34675000, "verified": true, "priceDuration": "sell", "sizeMin": NaN, "furnishing": "PARTLY", "description": "Discover elegance and modern living in Orla by Ominayat with this contemporary 3-bedroom apartment, seamlessly blending sophistication and comfort. The spacious living area integrates with the dining space, adorned with high-quality finishes, creating a visually pleasing environment.\\n\\nThe gourmet kitchen, a chef&#39;s delight, features top-of-the-line appliances and ample storage. Three generously sized bedrooms offer a peaceful retreat, flooded with natural light and breathtaking views of the Orla landscape. The master bedroom, with an en-suite bathroom, provides a private oasis for relaxation, while additional bedrooms share a well-app

In [17]:
x["value"].to_dict(orient="records")

[{'title': 'Dorchester Managed 3BR Waterfront Resale',
  'displayAddress': 'Orla by Omniyat, Palm Jumeirah, Dubai',
  'bathrooms': 4.0,
  'bedrooms': 3.0,
  'addedOn': '2024-07-29 08:02:12+00:00',
  'type': 'Residential for Sale',
  'price': 34675000,
  'verified': True,
  'priceDuration': 'sell',
  'sizeMin': nan,
  'furnishing': 'PARTLY',
  'description': 'Discover elegance and modern living in Orla by Ominayat with this contemporary 3-bedroom apartment, seamlessly blending sophistication and comfort. The spacious living area integrates with the dining space, adorned with high-quality finishes, creating a visually pleasing environment.\n\nThe gourmet kitchen, a chef&#39;s delight, features top-of-the-line appliances and ample storage. Three generously sized bedrooms offer a peaceful retreat, flooded with natural light and breathtaking views of the Orla landscape. The master bedroom, with an en-suite bathroom, provides a private oasis for relaxation, while additional bedrooms share a 

In [28]:
response2 = hi.chat("Retrieve properties located within a 2-mile radius from burj khalifa")

In [31]:
response2.value

,title,displayAddress,bathrooms,bedrooms,addedOn,type,price,verified,priceDuration,sizeMin,furnishing,description,latitude,longitude,City
0,Great ROI I High Floor I Creek View,"Binghatti Canal, Business Bay, Dubai",3.0,2.0,2024-08-14 12:02:53+00:00,Residential for Sale,2500000,True,sell,NaN,NO,MNA Properties is delighted to offer this apar...,25.186109,55.286156,Dubai
1,Luxurious | Panoramic views | Newly Renovated,"Building 19, City Walk, Dubai",4.0,2.0,2024-07-27 13:46:46+00:00,Residential for Sale,6000000,True,sell,NaN,YES,Discover luxurious living in this completely u...,25.208159,55.261874,Dubai
2,Genuine Resale | VOT | Burj and Fountain View,"Burj Royale, Downtown Dubai, Dubai",2.0,2.0,2024-07-17 05:12:16+00:00,Residential for Sale,4500000,True,sell,NaN,YES,Edge Realty Real Estate is proud to offer you ...,25.191644,55.280266,Dubai
3,0% Commission | Multiple options | Spacious,"Building 19, City Walk, Dubai",3.0,2.0,2024-07-24 10:41:51+00:00,Residential for Sale,3550000,True,sell,NaN,NO,Raas Real Estate offers you this one bedroom i...,25.208159,55.261874,Dubai
4,"For sale,2bed , view canal , Burj Khalifa","Binghatti Canal, Business Bay, Dubai",3.0,2.0,2024-08-10 14:47:45+00:00,Residential for Sale,2700000,False,sell,NaN,YES,"for sale 2 bed \nin Binghatti Canal, Business ...",25.186109,55.286156,Dubai
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,Zabeel Palace View | Prime Location | Downtown,"Downtown Views, Zabeel 2, Zabeel, Dubai",3.0,2.0,2024-08-05 12:50:50+00:00,Residential for Sale,3350000,False,sell,NaN,NO,"Located in the prestigious Downtown Views, thi...",25.202148,55.280633,Dubai
67,Exclusive | Canal View | Fully Furnished Unit,"Dolphin Tower, Business Bay, Dubai",1.0,NaN,2024-04-24 12:09:16+00:00,Residential for Sale,1100000,True,sell,NaN,YES,Re-selling this Exclusive Studio apartment wit...,25.183086,55.283063,Dubai
68,INVESTORS DEAL | FULLY FURNISHED | PRIME LOCATION,"Mayfair Residency, Business Bay, Dubai",1.0,1.0,2024-08-02 07:42:43+00:00,Residential for Sale,999999,True,sell,NaN,YES,Banke International properties is proud to pre...,25.181769,55.277677,Dubai
69,Burj Khalifa view | Fully Furnished | Handover...,"Nobles Tower, Business Bay, Dubai",2.0,1.0,2024-07-03 17:15:15+00:00,Residential for Sale,1700000,False,sell,NaN,YES,A luxurious residential development by Tiger P...,25.184624,55.290330,Dubai


In [32]:
len(response.value)

0

In [10]:
def main_agent(query: str):
    try:
        # Step 1: Relevance Check
        if not is_uae_real_estate_query(query):
            return {"type": "output", "data": "This is an irrelevant question to UAE property."}

        # Step 2: Extract data intent (what data exactly is needed)
        data_intent = extract_data_intent(query)
        if isinstance(data_intent, dict) and data_intent.get("error"):
            return data_intent

        # Step 3: Use data intent to query/extract data safely
        data = safe_dataframe_tool(data_intent)
        if isinstance(data, dict) and data.get("error"):
            return data

        # Step 4: Goal classification (what user wants to do with data)
        action = llm_classifier(query)
        if action not in {"output", "plot_stats", "geospatial_plot"}:
            return {"error": f"Unknown action '{action}' from classifier."}

        # Step 5: Action execution
        if action == "output":
            return {"type": "data", "data": data}

        elif action == "plot_stats":
            result = visualize_tool.invoke({"data": data, "query": query})
            return {"type": "plot", **result}

        elif action == "geospatial_plot":
            html_content = generate_google_maps_html(data)
            return {"type": "html", "content": html_content}

    except Exception as e:
        return {"error": f"Unexpected error: {str(e)}"}


In [9]:
main_agent("show properties on map that are located no further than 2 miles form Dubai Eye")

NameError: name 'main_agent' is not defined

In [4]:
safe_dataframe_tool("Extract properties data located within a 2-mile radius from Dubai Eye.")

                                                 title  \
0                  Great ROI I High Floor I Creek View   
33       Luxurious | Panoramic views | Newly Renovated   
35       Genuine Resale | VOT | Burj and Fountain View   
38         0% Commission | Multiple options | Spacious   
82           For sale,2bed , view canal , Burj Khalifa   
..                                                 ...   
502     Zabeel Palace View | Prime Location | Downtown   
516      Exclusive | Canal View | Fully Furnished Unit   
521  INVESTORS DEAL | FULLY FURNISHED | PRIME LOCATION   
526  Burj Khalifa view | Fully Furnished | Handover...   
547                 VACANT / ALL BRANDNEW / GUARANTEED   

                              displayAddress  bathrooms  bedrooms  \
0       Binghatti Canal, Business Bay, Dubai          3         2   
33             Building 19, City Walk, Dubai          4         2   
35        Burj Royale, Downtown Dubai, Dubai          2         2   
38             Building 19,

[{'title': 'Great ROI I High Floor I Creek View',
  'displayAddress': 'Binghatti Canal, Business Bay, Dubai',
  'bathrooms': 3,
  'bedrooms': 2,
  'addedOn': Timestamp('2024-08-14 12:02:53+0000', tz='UTC'),
  'type': 'Residential for Sale',
  'price': 2500000,
  'verified': True,
  'priceDuration': 'sell',
  'sizeMin': nan,
  'furnishing': 'NO',
  'description': 'MNA Properties is delighted to offer this apartment located in the well-esteemed Binghatti Canal, situated in the heart of Business Bay, Dubai, UAE. The 2BR apartment with the most attractive layout based on the high floor and offers the breathtaking panoramic view of Business Bay Canal; near Burj Khalifa.\n\nTo arrange a viewing or to find out more, please contact Amir via phone or Whatsapp on +971581684981 or email at amir.m@mnaproperties.ae\n\nProperty Features:\n2 Bedrooms\n2 Bathrooms + Guest room\nSize 1,323 sq. ft.\n2 Balconies\nPrivate Jacuzzi / Small Pool\nCanal View\n\nCan be sold Unfurnished as well\n\nBinghatti Can

In [ ]:
DATA_DIR = r"../data/uae_real_estate_2024_geo_ready2.parquet"
df =  pd.read_parquet(DATA_DIR)

In [5]:
df.to_csv(r"../data/uae_real_estate_2024_geo_ready2.csv", index=False)

In [6]:
import pandasai as pai

companies = pai.create(
  path="my-org/properties2",
  df=pai.read_csv("../data/uae_real_estate_2024_geo_ready2.csv"),
  description="UAE properties dataset 2"
)

Dataset saved successfully to path: my-org/properties2


In [7]:
result = companies.chat("Retrieve properties located within a 2-mile radius of Dubai Eye.")

In [8]:
result

DataFrameResponse(type='dataframe', value=                                                 title  \
27            Dorchester Managed 3BR Waterfront Resale   
86   New to Market | D type | Royal Atlantis And Ma...   
92             1BR I Great Size I Vacant I Well Priced   
179            Exclusive | Highest Quality | Brand new   
193     Luxurious Neighbourhood | Hot Offer | Sea View   
214     Genuine Listing | One Bedroom | Largest Layout   
226  YODEZEEN INTERIOR, DOUBLE HEIGHT LIVING, SEA VIEW   
230                  VACANT | Large terrace | Upgraded   
296      Prime Location | Furnished Studio | Palm View   
331         Studio with stunning Sea View | Palm Tower   
333          Marina View | Vacant | Stunning Townhouse   
334        Upgraded Kitchen | Waterfront | Multi-level   
405   PALM JUMEIRAH| UPGRADED | PRIVATE BEACH ACCESS |   
413   Exquisite | Vacant Now | Marina Views |Best Deal   

                                        displayAddress  bathrooms  bedrooms  \
27      

In [5]:
main_agent("show properties on map that are located no further than 2 miles form Dubai Eye")

/tmp/ipykernel_137746/4004079678.py:13: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  data = safe_dataframe_tool(data_intent)


                                                 title  \
0                  Great ROI I High Floor I Creek View   
33       Luxurious | Panoramic views | Newly Renovated   
35       Genuine Resale | VOT | Burj and Fountain View   
38         0% Commission | Multiple options | Spacious   
82           For sale,2bed , view canal , Burj Khalifa   
..                                                 ...   
502     Zabeel Palace View | Prime Location | Downtown   
516      Exclusive | Canal View | Fully Furnished Unit   
521  INVESTORS DEAL | FULLY FURNISHED | PRIME LOCATION   
526  Burj Khalifa view | Fully Furnished | Handover...   
547                 VACANT / ALL BRANDNEW / GUARANTEED   

                              displayAddress  bathrooms  bedrooms  \
0       Binghatti Canal, Business Bay, Dubai          3         2   
33             Building 19, City Walk, Dubai          4         2   
35        Burj Royale, Downtown Dubai, Dubai          2         2   
38             Building 19,

Traceback (most recent call last):
  File "/home/zcemg08/projects/chatbot/.chatbot_env/lib/python3.12/site-packages/pandasai/pipelines/chat/generate_chat_pipeline.py", line 335, in run
    ).run(input)
      ^^^^^^^^^^
  File "/home/zcemg08/projects/chatbot/.chatbot_env/lib/python3.12/site-packages/pandasai/pipelines/pipeline.py", line 137, in run
    raise e
  File "/home/zcemg08/projects/chatbot/.chatbot_env/lib/python3.12/site-packages/pandasai/pipelines/pipeline.py", line 101, in run
    step_output = logic.execute(
                  ^^^^^^^^^^^^^^
  File "/home/zcemg08/projects/chatbot/.chatbot_env/lib/python3.12/site-packages/pandasai/pipelines/chat/code_execution.py", line 113, in execute
    raise e
  File "/home/zcemg08/projects/chatbot/.chatbot_env/lib/python3.12/site-packages/pandasai/pipelines/chat/code_execution.py", line 85, in execute
    result = self.execute_code(code_to_run, code_context)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/zcemg08/

geospatial_plot


{'type': 'html',
 'content': '<html><body><h1>No locations provided</h1></body></html>'}